In [1]:
library(readr)
library(tidyverse)
library("qiime2R")
library("phyloseq")
library("microbiome")

ERROR: Error in library(readr): there is no package called 'readr'


In [ ]:
setwd('/Users/alchemist/Aakash/BonnGut/Microbiome_pipeline')
workdir = getwd()

In [ ]:
#
pseq<-qza_to_phyloseq(
    features=paste(workdir,"/feature_table/table.qza",sep=""),
    tree=paste(workdir,"/phylo_tree/rooted_tree.qza",sep=""),
         paste(workdir,"/taxonomy/tax_ezbiocloud.qza",sep=""),
    paste(workdir,"/metadata/metadata.tsv",sep="")
    )
#
feature_table = otu_data; 
sample_var    = "Sample.ID"; 
group_var     = "" ; 
main_var      = "" ; 
rand_formula  = "" ; 
adj_formula   = ""
#
out_cut       = 0.05; 
zero_cut      = 0.90; 
lib_cut       = 0; 
neg_lb        = TRUE; 
alpha         = 0.05 #sigificance threshold

In [ ]:
############################################ Differential Analysis at Phylum Level ###########################
#
otu_data = aggregate_taxa(pseq, "Phylum")
otu_data = otu_table(otu_data) #otu_table(ps)
# Step 1: Data preprocessing
source("/Users/alchemist/Aakash/BonnGut/gut_brain_project/scripts/ancom_v2_scripts/ancom_v2.1.R")
prepro = feature_table_pre_process(feature_table, meta_data, sample_var, group_var, 
                                   out_cut, zero_cut, lib_cut, neg_lb)
feature_table = prepro$feature_table # Preprocessed feature table
meta_data = prepro$meta_data # Preprocessed metadata
struc_zero = prepro$structure_zeros # Structural zero info

# # Step 2: ANCOM
t_start = Sys.time()
res_phylum = ANCOM(feature_table, meta_data, struc_zero, main_var, p_adj_method, 
            alpha, adj_formula, rand_formula)
t_end = Sys.time()
t_run = t_end - t_start

# Step 3: Volcano Plot

# Number of taxa except structural zeros
n_taxa = ifelse(is.null(struc_zero), nrow(feature_table), sum(apply(struc_zero, 1, sum) == 0))
# Cutoff values for declaring differentially abundant taxa
cut_off = c(0.9 * (n_taxa -1), 0.8 * (n_taxa -1), 0.7 * (n_taxa -1), 0.6 * (n_taxa -1))
names(cut_off) = c("detected_0.9", "detected_0.8", "detected_0.7", "detected_0.6")

# Annotation data
dat_ann = data.frame(x = min(res$fig$data$x), y = cut_off["detected_0.7"], label = "W[0.7]")

fig = res_phylum$fig +  
  geom_hline(yintercept = cut_off["detected_0.7"], linetype = "dashed") + 
  geom_text(data = dat_ann, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = "orange", parse = TRUE)
fig  

In [ ]:
############################################ Differential Analysis at Genus Level ###########################
pseq.genus = tax_glom(pseq, taxrank="Genus") 
otu_data <- core(pseq.genus, detection = 0.0001, prevalence = .5) #default filtering
#
otu_data = otu_table(otu_data) #otu_table(ps)
# Step 1: Data preprocessing
source("/Users/alchemist/Aakash/BonnGut/gut_brain_project/scripts/ancom_v2_scripts/ancom_v2.1.R")
prepro = feature_table_pre_process(feature_table, meta_data, sample_var, group_var, 
                                   out_cut, zero_cut, lib_cut, neg_lb)
feature_table = prepro$feature_table # Preprocessed feature table
meta_data = prepro$meta_data # Preprocessed metadata
struc_zero = prepro$structure_zeros # Structural zero info

# # Step 2: ANCOM
t_start = Sys.time()
res_genus = ANCOM(feature_table, meta_data, struc_zero, main_var, p_adj_method, 
            alpha, adj_formula, rand_formula)
t_end = Sys.time()
t_run = t_end - t_start

# Step 3: Volcano Plot

# Number of taxa except structural zeros
n_taxa = ifelse(is.null(struc_zero), nrow(feature_table), sum(apply(struc_zero, 1, sum) == 0))
# Cutoff values for declaring differentially abundant taxa
cut_off = c(0.9 * (n_taxa -1), 0.8 * (n_taxa -1), 0.7 * (n_taxa -1), 0.6 * (n_taxa -1))
names(cut_off) = c("detected_0.9", "detected_0.8", "detected_0.7", "detected_0.6")

# Annotation data
dat_ann = data.frame(x = min(res$fig$data$x), y = cut_off["detected_0.7"], label = "W[0.7]")

fig = res_genus$fig +  
  geom_hline(yintercept = cut_off["detected_0.7"], linetype = "dashed") + 
  geom_text(data = dat_ann, aes(x = x, y = y, label = label), 
            size = 4, vjust = -0.5, hjust = 0, color = "orange", parse = TRUE)
fig  